# Times 분할 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train = pd.read_csv('train_features.csv')
train_label = pd.read_csv('train_labels.csv')
test= pd.read_csv('test_features.csv')
submission = pd.read_csv('sample_submission.csv')

pd.set_option('display.max_columns', None)

In [2]:
train_ids = train.id.nunique()
test_ids = test.id.nunique()

In [3]:
train_df = [] 
for i in range(train_ids):
    train_df.append(train.iloc[600*i : 600*i + 600])
    
test_df = []
for i in range(test_ids):
    test_df.append(test.iloc[600*i : 600*i + 600 ])

In [4]:
def mad(x):
    return np.median(np.absolute(x - np.median(x, axis=0)), axis=0)

def energy(x): # enengy
    x = np.array(x)
    return np.sum(np.square(x)) / len(x)

def iqr(x):
    return np.subtract(*np.percentile(x, [75, 25]))

def entropy(x):
    p = x/x.sum()
    return (-p*np.log2(p)).sum()

def skewness(x):
    from scipy.stats import skew 
    return skew(x)

def kurtosis(x):
    from scipy.stats import kurtosis 
    return kurtosis(x)

def angleBet(u, v):
    from numpy import (array, dot, arccos, clip)
    from numpy.linalg import norm

    c = dot(u,v)/norm(u)/norm(v) # -> cosine of the angle
    angle = arccos(clip(c, -1, 1)) # if you really want the angle
    
    return angle

In [5]:
# times 별 집계 계산 함수 features 추가 
def splitTimes(df, Times):
    
    ids = len(df)
    df_t = pd.DataFrame()
    
    for j in range(ids):
        df_agg = pd.DataFrame()
        for i in range(600//Times):
            gb = df[j].iloc[i*Times : i*Times + Times].drop('time', axis = 1, inplace =False)
            tmp = gb.groupby('id').agg([np.min, np.max, np.std, np.mean, 
                                  mad, energy, iqr, entropy, skewness, kurtosis ])
            
            
            df_agg = pd.concat([df_agg, tmp], axis = 1)
            
        df_t = df_t.append(df_agg)
        
    return df_t

In [6]:
def addAngle(dfs):
    # test x, y, z 각 features 간의 angle 
    angle_acc_xy = []
    angle_acc_xz = []
    angle_acc_yz = [] 
    angle_gy_xy = [] 
    angle_gy_xz = [] 
    angle_gy_yz = [] 

    for i in range(len(dfs)):
    # acc x, y, z
        a_xy = angleBet(dfs[i].acc_x, dfs[i].acc_y)
        a_xz = angleBet(dfs[i].acc_x, dfs[i].acc_z)
        a_yz = angleBet(dfs[i].acc_y, dfs[i].acc_z)
        angle_acc_xy = np.append(angle_acc_xy, a_xy)
        angle_acc_xz = np.append(angle_acc_xz, a_xz)
        angle_acc_yz = np.append(angle_acc_yz, a_yz)
    
    # gy x, y, z
        g_xy = angleBet(dfs[i].gy_x, dfs[i].gy_y)
        g_xz = angleBet(dfs[i].gy_x, dfs[i].gy_z)
        g_yz = angleBet(dfs[i].gy_y, dfs[i].gy_z)
        angle_gy_xy = np.append(angle_gy_xy, g_xy)
        angle_gy_xz = np.append(angle_gy_xz, g_xz) 
        angle_gy_yz = np.append(angle_gy_yz, g_yz)
    
    
    angles = {'angle_acc_xy' : angle_acc_xy, 
         'angle_acc_yz' : angle_acc_yz,
         'angle_acc_xz' : angle_acc_xz,
         'angle_gy_xy' : angle_gy_xy,
         'angle_gy_yz': angle_gy_yz,
         'angle_gy_xz': angle_gy_xz}

    angles = pd.DataFrame(angles)
    
    return angles

In [8]:
train_200 = splitTimes(train_df, 200)
test_200 = splitTimes(test_df, 200)

In [12]:
train_ang = addAngle(train_df)
test_ang = addAngle(test_df)

In [15]:
test_200.reset_index()

id     acc_x                                                    \
               amin      amax       std      mean       mad    energy   
0    3125 -1.564000 -0.275446  0.259584 -0.989584  0.120941  1.046324   
1    3126 -1.015893  0.444149  0.326146 -0.536384  0.167323  0.393547   
2    3127 -0.036775  0.705013  0.098782  0.476207  0.060676  0.236482   
3    3128 -1.045889  0.171789  0.486688 -0.527978  0.181927  0.514441   
4    3129 -1.691371 -0.027340  0.314804 -0.916738  0.197827  0.939015   
..    ...       ...       ...       ...       ...       ...       ...   
777  3902 -1.601650 -0.328758  0.200209 -0.963907  0.084285  0.968999   
778  3903 -1.632634 -0.228192  0.270186 -0.910374  0.168727  0.901416   
779  3904 -1.302081 -0.498971  0.156574 -0.945427  0.102390  0.918226   
780  3905  0.536568  1.330448  0.173854  0.946841  0.139266  0.926581   
781  3906 -1.266320 -0.508193  0.158602 -0.905961  0.086000  0.845794   

                                                acc_y                      \
          iqr   entropy  skewness  kurtosis      amin      amax       std   
0    0.320468  7.589913  0.532268  0.009852 -0.239870  0.172556  0.075952   
1    0.335195  7.605180  1.128741  0.862755  0.128756  1.708743  0.256943   
2    0.122503  7.611813 -0.715166  2.787755  0.493329  1.253612  0.187961   
3    1.056222  7.393656  0.406481 -1.643580 -0.780307 -0.258476  0.175382   
4    0.388170  7.545015  0.373933  0.523476 -0.860883  0.813046  0.305013   
..        ...       ...       ...       ...       ...       ...       ...   
777  0.163035  7.611883 -0.143001  2.128712  0.045032  1.608399  0.258891   
778  0.352625  7.572950  0.481444  0.361092 -0.509494  0.830672  0.283284   
779  0.215959  7.623570  0.268335  0.002422 -0.443533  0.422296  0.164605   
780  0.279976  7.619539  0.128070 -0.773482 -0.580191 -0.024187  0.113886   
781  0.171877  7.620834  0.470425  0.249757 -0.766078  0.071348  0.176304   

                                                                            \
         mean       mad    energy       iqr    entropy  skewness  kurtosis   
0   -0.020038  0.053365  0.006141  0.108720  10.913396 -0.338745 -0.051155   
1    0.787744  0.181598  0.686230  0.357134   7.567202  0.504753  0.704358   
2    0.908842  0.142842  0.861147  0.303228   7.611955 -0.333228 -0.999124   
3   -0.504071  0.155203  0.284693  0.355163   7.554239  0.012497 -1.615704   
4    0.012573  0.163772  0.092726  0.304159  25.851063  0.369949  0.511721   
..        ...       ...       ...       ...        ...       ...       ...   
777  0.523597  0.093311  0.340843  0.178483   7.484812  1.622305  3.719704   
778  0.295785  0.199531  0.167338  0.431542   7.542326 -0.229165 -0.539575   
779 -0.014616  0.079963  0.027173  0.160319  18.682840  0.065082  0.398375   
780 -0.349926  0.078930  0.135353  0.152578   7.557757  0.385245  0.036016   
781 -0.275239  0.124695  0.106684  0.255285   7.345838 -0.250055 -0.420364   

        acc_z                                                              \
         amin      amax       std      mean       mad    energy       iqr   
0   -0.561128  0.151487  0.163658 -0.252037  0.129743  0.090173  0.246856   
1   -0.856899 -0.079464  0.145377 -0.450567  0.097195  0.224039  0.207718   
2   -0.409663  0.162760  0.133286 -0.157099  0.072969  0.042356  0.188567   
3   -0.227997  0.702574  0.349411  0.264410  0.334693  0.191390  0.722245   
4   -0.631258  0.952330  0.251089  0.167999  0.094178  0.090954  0.187337   
..        ...       ...       ...       ...       ...       ...       ...   
777 -0.562072  0.237183  0.152668 -0.144141  0.083124  0.043968  0.178314   
778 -0.573579  0.521008  0.231005 -0.099253  0.137089  0.062948  0.303479   
779 -0.332901  0.319565  0.149771  0.018317  0.107814  0.022655  0.217199   
780 -0.703792  0.084615  0.167434 -0.335912  0.129807  0.140731  0.249838   
781 -0.375468  0.621691  0.183957 -0.044421  0.105408  0.035644  0.248939   

                                      

In [16]:
train_200.columns = ['_'.join(col) for col in train_200.columns]
test_200.columns = ['_'.join(col) for col in test_200.columns]

In [17]:
gb_train_df = pd.merge(train_200, train_ang, left_index = True,right_index=True)
gb_test_df = pd.merge(test_200.reset_index(), test_ang, left_index =True, right_index = True)
gb_test_df.set_index('id', inplace = True)

In [8]:
y = train_label['label']

In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(gb_train_df, y, test_size = 0.2,
                                                   random_state = 42)
evals = [(X_test.values, y_test)]

In [9]:
import time
import datetime
from sklearn.metrics import log_loss

In [24]:
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.05,
                   max_depth = 8, objective = 'multi:softprob', random_state = 0)

In [ ]:
start = time.time()
xgb.fit(X_train.values, y_train, eval_metric ='mlogloss',
       eval_set = evals, early_stopping_rounds = 100)
sec = time.time() -start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

In [26]:
y_pred = xgb.predict_proba(X_test.values)
print('예상 log loss : ', log_loss(y_test, y_pred))

예상 log loss :  0.8974731564537156


In [ ]:
start = time.time()
xgb.fit(gb_train_df.values, y, eval_metric ='mlogloss',
       eval_set = evals, early_stopping_rounds = 100)
sec = time.time() - start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

In [10]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 0)

In [34]:
gb_train_df.shape

(3125, 186)

In [35]:
X_over, y_over = smote.fit_sample(gb_train_df.values, y )

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size = 0.2,
                                                   random_state = 42)

In [39]:
evals = [(X_test, y_test)]

In [40]:
start = time.time()
xgb.fit(X_train, y_train, eval_metric ='mlogloss',
       eval_set = evals, early_stopping_rounds = 100)
sec = time.time() -start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

[0]	validation_0-mlogloss:2.94354
[1]	validation_0-mlogloss:2.56100
[2]	validation_0-mlogloss:2.29448
[3]	validation_0-mlogloss:2.08771
[4]	validation_0-mlogloss:1.91950
[5]	validation_0-mlogloss:1.77723
[6]	validation_0-mlogloss:1.65404
[7]	validation_0-mlogloss:1.54567
[8]	validation_0-mlogloss:1.44935
[9]	validation_0-mlogloss:1.36291
[10]	validation_0-mlogloss:1.28454
[11]	validation_0-mlogloss:1.21293
[12]	validation_0-mlogloss:1.14708
[13]	validation_0-mlogloss:1.08625
[14]	validation_0-mlogloss:1.02974
[15]	validation_0-mlogloss:0.97734
[16]	validation_0-mlogloss:0.92850
[17]	validation_0-mlogloss:0.88269
[18]	validation_0-mlogloss:0.83988
[19]	validation_0-mlogloss:0.79961
[20]	validation_0-mlogloss:0.76182
[21]	validation_0-mlogloss:0.72600
[22]	validation_0-mlogloss:0.69225
[23]	validation_0-mlogloss:0.66039
[24]	validation_0-mlogloss:0.63028
[25]	validation_0-mlogloss:0.60164
[26]	validation_0-mlogloss:0.57450
[27]	validation_0-mlogloss:0.54879
[28]	validation_0-mlogloss:0.5

In [41]:
y_pred = xgb.predict_proba(X_test)
print('예상 log loss : ', log_loss(y_test, y_pred))

예상 log loss :  0.004891874810103652


In [43]:
start = time.time()
xgb.fit(X_over, y_over, eval_metric ='mlogloss', eval_set = evals,
        early_stopping_rounds = 100)
sec = time.time() -start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

[0]	validation_0-mlogloss:2.88755
[1]	validation_0-mlogloss:2.50113
[2]	validation_0-mlogloss:2.23287
[3]	validation_0-mlogloss:2.02401
[4]	validation_0-mlogloss:1.85425
[5]	validation_0-mlogloss:1.71138
[6]	validation_0-mlogloss:1.58788
[7]	validation_0-mlogloss:1.47957
[8]	validation_0-mlogloss:1.38307
[9]	validation_0-mlogloss:1.29661
[10]	validation_0-mlogloss:1.21820
[11]	validation_0-mlogloss:1.14654
[12]	validation_0-mlogloss:1.08086
[13]	validation_0-mlogloss:1.02021
[14]	validation_0-mlogloss:0.96421
[15]	validation_0-mlogloss:0.91206
[16]	validation_0-mlogloss:0.86341
[17]	validation_0-mlogloss:0.81799
[18]	validation_0-mlogloss:0.77560
[19]	validation_0-mlogloss:0.73574
[20]	validation_0-mlogloss:0.69835
[21]	validation_0-mlogloss:0.66319
[22]	validation_0-mlogloss:0.63011
[23]	validation_0-mlogloss:0.59881
[24]	validation_0-mlogloss:0.56929
[25]	validation_0-mlogloss:0.54149
[26]	validation_0-mlogloss:0.51515
[27]	validation_0-mlogloss:0.49020
[28]	validation_0-mlogloss:0.4

In [44]:
submission.iloc[:,1:] = xgb.predict_proba(gb_test_df.values)
submission.to_csv('submission파일/over_xgb_200T_aggUA.csv', index = False)

제출 결과값 : 0.8043752923

In [14]:
train_150 = splitTimes(train_df, 150)
test_150 = splitTimes(test_df, 150)
train_150.columns = ['_'.join(col) for col in train_150.columns]
test_150.columns = ['_'.join(col) for col in test_150.columns]
train_ang = addAngle(train_df)
test_ang = addAngle(test_df)

gb_train_df = pd.merge(train_150, train_ang, left_index = True,right_index=True)
gb_test_df = pd.merge(test_150.reset_index(), test_ang, left_index =True, right_index = True)
gb_test_df.set_index('id', inplace = True)

X_over, y_over = smote.fit_sample(gb_train_df.values, y )

X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size = 0.2,
                                                   random_state = 42)

In [15]:
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.08,
                   max_depth = 8, objective = 'multi:softprob', random_state = 0)

evals = [(X_test, y_test)]

start = time.time()
xgb.fit(X_over, y_over, eval_metric ='mlogloss', eval_set = evals,
        early_stopping_rounds = 100)
sec = time.time() -start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

[0]	validation_0-mlogloss:2.20817
[1]	validation_0-mlogloss:1.88148
[2]	validation_0-mlogloss:1.64815
[3]	validation_0-mlogloss:1.46535
[4]	validation_0-mlogloss:1.31586
[5]	validation_0-mlogloss:1.18913
[6]	validation_0-mlogloss:1.07972
[7]	validation_0-mlogloss:0.98387
[8]	validation_0-mlogloss:0.89983
[9]	validation_0-mlogloss:0.82446
[10]	validation_0-mlogloss:0.75694
[11]	validation_0-mlogloss:0.69613
[12]	validation_0-mlogloss:0.64099
[13]	validation_0-mlogloss:0.59090
[14]	validation_0-mlogloss:0.54519
[15]	validation_0-mlogloss:0.50332
[16]	validation_0-mlogloss:0.46516
[17]	validation_0-mlogloss:0.43001
[18]	validation_0-mlogloss:0.39770
[19]	validation_0-mlogloss:0.36802
[20]	validation_0-mlogloss:0.34070
[21]	validation_0-mlogloss:0.31553
[22]	validation_0-mlogloss:0.29228
[23]	validation_0-mlogloss:0.27079
[24]	validation_0-mlogloss:0.25088
[25]	validation_0-mlogloss:0.23253
[26]	validation_0-mlogloss:0.21554
[27]	validation_0-mlogloss:0.19982
[28]	validation_0-mlogloss:0.1

In [16]:
submission.iloc[:,1:] = xgb.predict_proba(gb_test_df.values)
submission.to_csv('submission파일/over_xgb_150T_aggUA.csv', index = False)

제출 결과값 : 0.8795788075